_Kristian M.P. Dashnaw, Software Engineering student at VIA University College, Horsens._


# __Assignment 3 - ADS Fall 2024__

The objective of this assignment is to create a weighted graph class. Include functions for adding and deleting vertices and edges, changing weights and iterating over all neighbors of a particular vertex.

Your code should look something like this:
  
    class Graph:
      def __init__(self, #add something here):
        #add something here
      def addVertex(self, #add something here):
        #add something here
      def addEdge(self, #add something here):
        #add something here
        
    #and so on
    

### __Deliverables:__

You are to hand in a <i>single</i> .ipynb containing:
1. Output (all cells must be run)
2. Your Graph class
3. Your instance of the graph above.


### __Design Choices:__
The solve the designated task of building a Graph ADT (Abstract Data Type), there are multiple recommended approaches, as described in Data Algorithms & Structures chapter 14. The most common include:
1. The __edge list__, which is a minimal approach to the Graph ADT where all edges are stored as an unordered list. Its performance is not good, and it provides no means of locating a particular edge, or a set of alle edges adjacent to a specific vertex, quickly.
2. The __adjacency list__, where separate lists are maintained for each vertex containing connected edges. This structure fulfils the requirements set by this assignment, allowing for identifying specific vertices and connected edges.
3. The __adjacency map__, which is similar to the adjacency list. Instead of using lists to store the edges connected to each vertex, this approach uses maps providing better performance. It also fulfills the requirements of this assignment.
4. The __adjacency matrix__, which uses a matrix structure to contain all vertices and connected edges, instead of using lists or maps. It allows for improved performance when getting each edge, but potential worse performance when inserting or removing vertices. It also fulfills the requirements of this assignment.



### __Chosen design:__ Adjacency Map

The adjacency map approach, described in option 3, is chosen as the base for this assignment. This choice is based upon the generally improved performance over option 1 and 2. Compared to option 4, I do not expect the written graphs to be dense especially often, which is why the improved performance the adjacency matrix offers while getting edges is negligible, while the worsened performance when inserting/deleting vertices are expected to be felt more often as a general user.

In [19]:
# Define the main Graph class, responsible for providing the Graph functionality.
# This is an adjacency map based Graph ADT.
# v = destination vertex
# u = origin vertex
# x = weight
class Graph:

    # Nested vertex class, used to represent all edges in this Graph ADT.
    class Vertex:
        __slots__ = '_element' # Will hold any destination vertex, once set.

        def __init__(self, x):
            self._element = x

        # Define a function to return the 'element' associated with this vertex (i.e. any associated destination vertex):
        def element(self):
            return self._element

        # A hash function is required, for custom classes to be represented as keys in dictionaries.
        def __hash__(self):
            return hash(id(self))

    # Nested edge class, used to represent all edges in this Graph ADT.
    class Edge:
        __slots__ = '_origin', '_destination', '_element'

        def __init__(self, u, v, x):
            self._origin = u        # Will hold the origin vertex for this edge.
            self._destination = v   # Will hold the destination vertex for this edge.
            self._element = x       # Will hold any weight element, for this edge (note: _element is chosen, for the added flexibility of the 'weight' being something else... maybe distance, time, etc.)

        # Returns a tuple, similar to dict, this edges key/value pair ("origin_vertex", "destination_vertex")
        def endpoints(self):
            return self._origin, self._destination

        # Returns the vertex which is at the opposite end of this edge (given edge ('A', 'B') if v='A' then return value is 'B'):
        def opposite(self, v):
            return self._destination if v is self._origin else self._origin

        # Returns any weight associated with this specific edge:
        def element(self):
            return self._element

        # A hash function is required, for custom classes to be represented as keys in dictionaries.
        def __hash__(self):
            return hash((self._origin, self._destination))


    # Define the Graph init method, which is run automatically when creating an instance (Also known as 'Constructor')
    def __init__(self, directed=False):
        self._outgoing = {}                                     # A dictionary containing lists of all outgoing edges (outgoing from a certain vertice, where vertice is the key and list of edges is the value)
        self._incoming = {} if directed else self._outgoing     # A dictionary containing lists of all incoming edges, if using a directed graph. Otherwise, works the same as the outgoing.

    # Returns TRUE if this Graph is declared as a directed graph, else returns FALSE if Graph is undirected.
    def is_directed(self):
        return self._incoming is not self._outgoing

    # Returns the total number of vertices objects added to this Graph
    def vertex_count(self):
        return len(self._outgoing)

    # Returns a list of all the vertice objects in this Graph
    def vertices(self):
        return self._outgoing.keys()

    # Returns the total number of edge objects added to this Graph
    def edge_count(self):
        total = sum(len(self._outgoing[v]) for v in self._outgoing)
        return total if self.is_directed() else total

    # Returns a list of all the edge objects in this Graph
    def edges(self):
        result = set( )
        for secondary_map in self._outgoing.values():
            result.update(secondary_map.values())
        return result

    # Returns a specific edge object
    def get_edge(self, u, v):
        return self._outgoing[u].get(v)

    # Calculates how many outgoing edges are directly connected to the specified vertex v.
    # If graph is non-directional it simply counts the number of incoming edges to any given vertex v.
    def degree(self, v, outgoing=True):
        adj = self._outgoing if outgoing else self._incoming
        return len(adj[v])

    # Returns a list of all edge objects either going out from the specified vertex v, or coming into - depending on the
    # boolean value of the 'outgoing' argument. If True, returns the edge objects originating from vertex v.
    def incident_edges(self, v, outgoing=True):
        adj = self._outgoing if outgoing else self._incoming
        for edge in adj[v].values():
            yield edge

    # Function to add a vertex object to the graph:
    def add_vertex(self, x = None):
        v = self.Vertex(x)
        self._outgoing[v] = {}
        if self.is_directed():
            self._incoming[v] = {}
        return v

    # Function to add an edge object to the graph:
    def add_edge(self, u, v, x=None):
        e = self.Edge(u, v, x)
        self._outgoing[u][v] = e
        self._incoming[v][u] = e
           
    # Function to delete a vertex (including any edges the vertices are part of):
    def remove_vertex(self, v):
        connectedEdgesOutgoing = self.incident_edges(v, True)
        for edge in connectedEdgesOutgoing:
            u = edge.opposite(v)
            del self._incoming[u][v]            # Removes the vertex from opposite end vertices' incoming map.
        del self._outgoing[v]                   # Removes the specified vertex from the outgoing vertices map.

        if self.is_directed():
            connectedEdgesIncoming = self.incident_edges(v, False)
            for edge in connectedEdgesIncoming: # Iterates through all incoming edges, connected to the specified vertex
                u = edge.opposite(v)
                del self._outgoing[u][v]        # Removes the vertex from opposite end vertices' outgoing map.
            del self._incoming[v]               # Removes the specified vertex from the incoming vertices map.
                 
    # Function to delete an edge, beginning in vertex u and ending in vertex v:
    def remove_edge(self, u, v):
        if self.is_directed():
            del self._outgoing[u][v]            # Removes the edge from the outgoing edge map.
        else:
            del self._outgoing[u][v]            # Removes the edge from the outgoing edge map.
            del self._incoming[v][u]            # Removes the edge from the incoming edge map, if graph is directed.

    # Function to change the weight of an edge, beginning in vertex u and ending in vertex v:
    def set_new_weight_on_edge(self, u, v, new_weight):
        self.remove_edge(u, v)
        self.add_edge(u,v, new_weight)

    # Function to display the current graph:
    def display_graph(self):
        for vertex, edges in self._outgoing.items():
            edges_str = {v._element: e._element for v, e in edges.items()}
            print(f"{vertex._element} -> {edges_str}")

## __Instantiating the Graph described in Assignment 3:__

Below we instantiate the Graph shown in Assignment 3. The expected visual representation, if our code above is written correctly, should be:

    A -> {'B': 2, 'C': 1, 'D': 7, 'E': 9}
    B -> {'A': 2, 'E': 1}
    C -> {'A': 1, 'F': 6, 'G': 12}
    D -> {'A': 7, 'E': 3, 'G': 2}
    E -> {'A': 9, 'B': 1, 'D': 3, 'G': 8, 'H': 7}
    F -> {'C': 6, 'G': 2}
    G -> {'C': 12, 'D': 2, 'E': 8, 'F': 2, 'H': 1}
    H -> {'E': 7, 'G': 1}

In [20]:
# Building the graph specified in Assignment 3:
weighedGraph = Graph(False)

# Add the vertices shown on the assignment 3 graph:
for vertex in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    weighedGraph.add_vertex(vertex)

# Build the mapping after all vertices are added:
vertices = {v.element(): v for v in weighedGraph.vertices()}

# Define the edges, visually shown on the assignment 3 graph:
edges = [
    ('A', 'B', 2), ('A', 'C', 1), ('A', 'D', 7), ('A', 'E', 9),
    ('B', 'A', 2), ('B', 'E', 1),
    ('C', 'A', 1), ('C', 'F', 6), ('C', 'G', 12),
    ('D', 'A', 7), ('D', 'E', 3), ('D', 'G', 2),
    ('E', 'B', 1), ('E', 'A', 9), ('E', 'D', 3), ('E', 'G', 8), ('E', 'H', 7),
    ('F', 'C', 6), ('F', 'G', 2),
    ('G', 'C', 12), ('G', 'D', 2), ('G', 'E', 8), ('G', 'F', 2), ('G', 'H', 1),
    ('H', 'E', 7), ('H', 'G', 1)]

# Add the edges shown on the assignment 3 graph:
for origin, destination, weight in edges:
    weighedGraph.add_edge(vertices[origin], vertices[destination], weight)

# Display the graph from Assignment 3, created using the Graph ADT written above:
weighedGraph.display_graph()

A -> {'B': 2, 'C': 1, 'D': 7, 'E': 9}
B -> {'A': 2, 'E': 1}
C -> {'A': 1, 'F': 6, 'G': 12}
D -> {'A': 7, 'E': 3, 'G': 2}
E -> {'A': 9, 'B': 1, 'D': 3, 'G': 8, 'H': 7}
F -> {'C': 6, 'G': 2}
G -> {'C': 12, 'D': 2, 'E': 8, 'F': 2, 'H': 1}
H -> {'E': 7, 'G': 1}


## __Testing the Graph ADT functions:__

To finalize this assignment, I have included a few brief tests of the remove and edit methods required in this assigment.
These showcase how the graph properly changes depending on the requested manipulation function

In [21]:
# Test removing an edge:
weighedGraph.remove_edge(vertices['A'], vertices['B'])

# Display updated graph:
weighedGraph.display_graph()

# Notice how the 'A' -> 'B' edge has been removed, as well as how the 'B' -> 'A' edge (since the declared graph is non-directional).

A -> {'C': 1, 'D': 7, 'E': 9}
B -> {'E': 1}
C -> {'A': 1, 'F': 6, 'G': 12}
D -> {'A': 7, 'E': 3, 'G': 2}
E -> {'A': 9, 'B': 1, 'D': 3, 'G': 8, 'H': 7}
F -> {'C': 6, 'G': 2}
G -> {'C': 12, 'D': 2, 'E': 8, 'F': 2, 'H': 1}
H -> {'E': 7, 'G': 1}


In [22]:
# Test removing a vertex:
weighedGraph.remove_vertex(vertices['E'])

# Display updated graph:
weighedGraph.display_graph()

# Notice how the 'E' vertex has been removed from the entire Graph.

A -> {'C': 1, 'D': 7}
B -> {}
C -> {'A': 1, 'F': 6, 'G': 12}
D -> {'A': 7, 'G': 2}
F -> {'C': 6, 'G': 2}
G -> {'C': 12, 'D': 2, 'F': 2, 'H': 1}
H -> {'G': 1}


In [23]:
# Test changing edge weight:
weighedGraph.set_new_weight_on_edge(vertices['A'], vertices['C'], 50)

# Display updated graph:
weighedGraph.display_graph()

# Notice how the 'A' -> 'C' edge weight has been changed to 50, as well as how the 'C' -> 'A' edge also increased in weight (since the declared graph is non-directional).

A -> {'D': 7, 'C': 50}
B -> {}
C -> {'F': 6, 'G': 12, 'A': 50}
D -> {'A': 7, 'G': 2}
F -> {'C': 6, 'G': 2}
G -> {'C': 12, 'D': 2, 'F': 2, 'H': 1}
H -> {'G': 1}
